In [88]:
import pandas as pd
import numpy as np
import os
import openai

In [74]:
input_path = "results.csv"
dir_path = os.path.abspath('')
data_path = f"{dir_path}/../{input_path}"
data_df = pd.read_csv(data_path, header=0, index_col=0)
data_df

,param,convo_num,summary,classification
0,1,0,"You enjoy the way smoking calms you, but disli...",1
1,1,1,You enjoy the feeling and physical act of smok...,1
2,1,2,You enjoy the relief that smoking provides you...,1
3,1,3,You enjoy the smell of cigarettes but dislike ...,1
4,1,4,You enjoy the taste of tobacco and are looking...,1
...,...,...,...,...
595,2,0,You express that you enjoy the feeling of rela...,1
596,2,1,You and the bot discuss the things you like an...,0
597,2,2,You express your conflicted feelings about smo...,0
598,2,3,You are trying to stop smoking when you are dr...,1


In [81]:
cols = data_df.columns

filtered_data = data_df.drop([cols[0], cols[1]], axis=1)
filtered_data.head()

,summary,classification
0,"You enjoy the way smoking calms you, but disli...",1
1,You enjoy the feeling and physical act of smok...,1
2,You enjoy the relief that smoking provides you...,1
3,You enjoy the smell of cigarettes but dislike ...,1
4,You enjoy the taste of tobacco and are looking...,1


In [82]:
# completion of 0 means the summary was poor, 1 means it satisfied the requirements
cols = filtered_data.columns
filtered_data = filtered_data.rename(columns={cols[0]: "prompt", cols[1]: "completion"})
filtered_data.head()

,prompt,completion
0,"You enjoy the way smoking calms you, but disli...",1
1,You enjoy the feeling and physical act of smok...,1
2,You enjoy the relief that smoking provides you...,1
3,You enjoy the smell of cigarettes but dislike ...,1
4,You enjoy the taste of tobacco and are looking...,1


In [83]:
#apparently you want 100 samples of each class when finetuning

filtered_data.groupby("completion").count()

,prompt
completion,
At,1
Before,1
In,1
Making,1
The,1
To,1
You,14
on,1
that,1


In [93]:
indexs_to_drop = filtered_data.index.difference(filtered_data[filtered_data["completion"].isin(['0', '1'])].index)
filtered_data.drop(indexs_to_drop, inplace=True)
filtered_data.groupby("completion").count()

,prompt
completion,
0,154
1,422


In [85]:
filtered_data = filtered_data.astype({"completion": str})

In [95]:
num_to_drop = 422-154
indexs_to_drop = np.random.choice(filtered_data[filtered_data["completion"]=='1'].index, 
                 num_to_drop, replace=False)
balanced_filtered_data = filtered_data.drop(indexs_to_drop)
balanced_filtered_data.groupby("completion").count()

,prompt
completion,
0,154
1,154


In [97]:
data_jsonl = balanced_filtered_data.to_json("data.jsonl", orient='records', lines=True)

In [98]:
!openai tools fine_tunes.prepare_data -f data.jsonl -q

Analyzing...

- Your file contains 308 prompt-completion pairs
- Based on your data it seems like you're trying to fine-tune a model for classification
- For classification, we recommend you try one of the faster and cheaper models, such as `ada`
- For classification, you can estimate the expected model performance by keeping a held out dataset, which is not used for training
- There are 2 duplicated prompt-completion sets. These are rows: [34, 70]
- Your data does not contain a common separator at the end of your prompts. Having a separator string appended to the end of the prompt makes it clearer to the fine-tuned model where the completion should begin. See https://beta.openai.com/docs/guides/fine-tuning/preparing-your-dataset for more detail and examples. If you intend to do open-ended generation, then you should leave the prompts empty
- The completion should start with a whitespace character (` `). This tends to produce better results due to the tokenization we use. See https://b

In [107]:
with open(f'{dir_path}/../openai_key.txt') as f:
        openai.api_key = f.read()  

In [108]:
openai.File.create(file=open("data_prepared_train.jsonl", "rb"),purpose='fine-tune')

<File file id=file-fm2TzbUhsk0emhioan6DoSgC at 0x7fa77e370860> JSON: {
  "bytes": 74639,
  "created_at": 1675763610,
  "filename": "file",
  "id": "file-fm2TzbUhsk0emhioan6DoSgC",
  "object": "file",
  "purpose": "fine-tune",
  "status": "uploaded",
  "status_details": null
}

In [109]:
openai.File.create(file=open("data_prepared_valid.jsonl", "rb"),purpose='fine-tune')

<File file id=file-y2zhT3jVDCteAGVT4wNV24al at 0x7fa77e36e180> JSON: {
  "bytes": 19928,
  "created_at": 1675763616,
  "filename": "file",
  "id": "file-y2zhT3jVDCteAGVT4wNV24al",
  "object": "file",
  "purpose": "fine-tune",
  "status": "uploaded",
  "status_details": null
}

In [110]:
openai.FineTune.create(training_file="file-fm2TzbUhsk0emhioan6DoSgC", 
                       validation_file="file-y2zhT3jVDCteAGVT4wNV24al", 
                       model="davinci", compute_classification_metrics=True,
                       classification_positive_class=" 0", suffix="summary-classifier-more-data")

<FineTune fine-tune id=ft-HRWFTlCJvjELOmeps0Q14aMz at 0x7fa78c1215e0> JSON: {
  "created_at": 1675763726,
  "events": [
    {
      "created_at": 1675763726,
      "level": "info",
      "message": "Created fine-tune: ft-HRWFTlCJvjELOmeps0Q14aMz",
      "object": "fine-tune-event"
    }
  ],
  "fine_tuned_model": null,
  "hyperparams": {
    "batch_size": null,
    "classification_positive_class": " 0",
    "compute_classification_metrics": true,
    "learning_rate_multiplier": null,
    "n_epochs": 4,
    "prompt_loss_weight": 0.01
  },
  "id": "ft-HRWFTlCJvjELOmeps0Q14aMz",
  "model": "davinci",
  "object": "fine-tune",
  "organization_id": "org-dqXnzwpBRx9JeWkSdgAJdRLI",
  "result_files": [],
  "status": "pending",
  "training_files": [
    {
      "bytes": 74639,
      "created_at": 1675763610,
      "filename": "file",
      "id": "file-fm2TzbUhsk0emhioan6DoSgC",
      "object": "file",
      "purpose": "fine-tune",
      "status": "processed",
      "status_details": null
    }
 

In [111]:
openai.FineTune.create(training_file="file-fm2TzbUhsk0emhioan6DoSgC", 
                       validation_file="file-y2zhT3jVDCteAGVT4wNV24al", 
                       model="ada", compute_classification_metrics=True,
                       classification_positive_class=" 0", suffix="summary-classifier-more-data")

<FineTune fine-tune id=ft-5QRGSbuSnvfSFhOnJpKc93q9 at 0x7fa7380c3f90> JSON: {
  "created_at": 1675763735,
  "events": [
    {
      "created_at": 1675763735,
      "level": "info",
      "message": "Created fine-tune: ft-5QRGSbuSnvfSFhOnJpKc93q9",
      "object": "fine-tune-event"
    }
  ],
  "fine_tuned_model": null,
  "hyperparams": {
    "batch_size": null,
    "classification_positive_class": " 0",
    "compute_classification_metrics": true,
    "learning_rate_multiplier": null,
    "n_epochs": 4,
    "prompt_loss_weight": 0.01
  },
  "id": "ft-5QRGSbuSnvfSFhOnJpKc93q9",
  "model": "ada",
  "object": "fine-tune",
  "organization_id": "org-dqXnzwpBRx9JeWkSdgAJdRLI",
  "result_files": [],
  "status": "pending",
  "training_files": [
    {
      "bytes": 74639,
      "created_at": 1675763610,
      "filename": "file",
      "id": "file-fm2TzbUhsk0emhioan6DoSgC",
      "object": "file",
      "purpose": "fine-tune",
      "status": "processed",
      "status_details": null
    }
  ],


In [116]:
openai.FineTune.list()

<OpenAIObject list at 0x7fa74808ebd0> JSON: {
  "data": [
    {
      "created_at": 1674343916,
      "fine_tuned_model": "davinci:ft-personal:summary-classifier-2023-01-22-02-04-52",
      "hyperparams": {
        "batch_size": 1,
        "classification_positive_class": "1",
        "compute_classification_metrics": true,
        "learning_rate_multiplier": 0.1,
        "n_epochs": 4,
        "prompt_loss_weight": 0.01
      },
      "id": "ft-STBuWrHnWV8iVrkSszXOjOH0",
      "model": "davinci",
      "object": "fine-tune",
      "organization_id": "org-dqXnzwpBRx9JeWkSdgAJdRLI",
      "result_files": [
        {
          "bytes": 17989,
          "created_at": 1674353096,
          "filename": "compiled_results.csv",
          "id": "file-nicqMJ2znGBVzmUUjkIZFB3f",
          "object": "file",
          "purpose": "fine-tune-results",
          "status": "processed",
          "status_details": null
        }
      ],
      "status": "succeeded",
      "training_files": [
        {


In [118]:
davinci_res_txt = openai.File.download(id="file-ZDNNrQlWBp8Fw3kUQzOcyLiW").decode("utf-8") 
ada_res_txt = openai.File.download(id="file-Is3hKa0jbxbZqTt4EKc7A7Im").decode("utf-8") 

In [119]:
with open("davinci_finetuning_results.txt", "w") as text_file:
    text_file.write(davinci_res_txt)

with open("ada_finetuning_results.txt", "w") as text_file:
    text_file.write(ada_res_txt)

In [130]:
ft_model = "davinci:ft-personal:summary-classifier-2023-01-22-02-04-52"
data = "You want to reduce the number of times you smoke a day, you say you plan to do this by being more motivated."
res = openai.Completion.create(model=ft_model, prompt=data, max_tokens=1)
res['choices'][0]['text']

'1'

## Notes from meeting

Redo the finetuning process with more data, try doubling the dataset. We could try bootstrapping the current classifier to label some of the data to save on time
Bootstrapping - 